# LightGBM(GPU) Install

In [ ]:
!git clone --recursive https://github.com/Microsoft/LightGBM
%cd /content/LightGBM
!mkdir build
!cmake -DUSE_GPU=1 #avoid ..
!make -j$(nproc)
!sudo apt-get -y install python-pip
!sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
%cd /content/LightGBM/python-package
!sudo python setup.py install --precompile

Cloning into 'LightGBM'...
remote: Enumerating objects: 26285, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 26285 (delta 36), reused 56 (delta 21), pack-reused 26199
Receiving objects: 100% (26285/26285), 19.04 MiB | 16.72 MiB/s, done.
Resolving deltas: 100% (19418/19418), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/external_libs/compute'...
remote: Enumerating objects: 21733, done.        
remote: Counting objects: 100% (5/5), done.        
remote: Compressi

# Setup

In [14]:
import os
# python global seed
os.environ['PYTHONHASHSEED'] = str(42)
# tensorflow seed (not working for GPU)
# os.environ['TF_DETERMINISTIC_OPS'] = '1'
# os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
# weight and bias
os.environ["WANDB_API_KEY"] = "b838b62906ab267778c6e05b913ba6c4a27699b2"

In [15]:
from google.colab import drive
drive.mount('/content/drive')
# # TextVectorization layer in tf 2.6 don't support "sparse=True" option which is used on TF-IDF
# # !pip install -q tensorflow==2.6.0
!pip install -q tensorflow==2.8
!pip install -q tensorflow-recommenders==0.6
# !pip install -q scann==1.2.3
# !pip install -q scann==1.2.6
# # !pip install -q tensorflow-datasets
!pip install -q tensorflow-addons
# !pip install -q tensorflow-hub
# # !pip install -q keras-tuner

# !pip install -q transformers

!pip install -q statsmodels
# !pip install -q xgboost
# !pip install -q lightgbm
# !pip install -q catboost
!pip install -q missingpy

!pip install -q optuna
!pip install -q wandb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
def config_missingpy():
    try:
        with open("/usr/local/lib/python3.7/site-packages/missingpy/knnimpute.py", 'r') as file:
        # read a list of lines into data
            data = file.readlines()
            # now change the 2nd line, note that you have to add a newline
            data[12] = 'from sklearn.neighbors._base import _check_weights\n'
            data[13] = 'from sklearn.neighbors._base import _get_weights\n'
        # and write everything back
        with open("/usr/local/lib/python3.7/site-packages/missingpy/knnimpute.py", 'w') as file:
            file.writelines(data)
    except:
        print("path error : /usr/local/lib/python3.7/site-packages/missingpy/knnimpute.py")
    try:
        with open("/usr/local/lib/python3.7/dist-packages/missingpy/knnimpute.py", 'r') as file:
        # read a list of lines into data
            data = file.readlines()
            # now change the 2nd line, note that you have to add a newline
            data[12] = 'from sklearn.neighbors._base import _check_weights\n'
            data[13] = 'from sklearn.neighbors._base import _get_weights\n'
        # and write everything back
        with open("/usr/local/lib/python3.7/dist-packages/missingpy/knnimpute.py", 'w') as file:
            file.writelines(data)
    except:
        print("path error : /usr/local/lib/python3.7/site-packages/missingpy/knnimpute.py")

In [17]:
import sys
import shutil
from glob import glob
import multiprocessing as mp
import gc
from pathlib import Path
from scipy import stats
from scipy.special import boxcox, softmax
from scipy import sparse
import itertools

from multiprocessing import cpu_count
import copy
import pickle
import warnings
from datetime import datetime, timedelta
from time import time, sleep, mktime
from matplotlib import font_manager as fm, rc, rcParams
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import re
import random as rnd
import psutil
from optuna import Trial, create_study
from optuna.samplers import TPESampler
import wandb

import numpy as np
from numpy import array, nan, random as np_rnd, where
from numpy import dot
from numpy.linalg import norm
import pandas as pd
from pandas import DataFrame as dataframe, Series as series, isna, read_csv
from pandas.tseries.offsets import DateOffset

from sklearn.model_selection import train_test_split as tts, StratifiedKFold, GroupKFold, GroupShuffleSplit, StratifiedGroupKFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler, KBinsDiscretizer
from sklearn import metrics
# from sklearn.compose import ColumnTransformer
config_missingpy(); from missingpy import MissForest
# from sklearn.impute import KNNImputer
# from sklearn.feature_extraction.text import TfidfVectorizer

# try:
#     # RAPIDS config
#     os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
#     os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
#     os.environ['CONDA_PREFIX'] = '/usr/local'
#     import cudf as cd
#     import cupy as cp
#     from cuml.cluster import KMeans
#     from cuml.neighbors import NearestNeighbors
#     from cuml.metrics.cluster import silhouette_score
# except:
#     print("RAPIDS Import ERROR")

# import xgboost as xgb
import lightgbm as lgb
# import catboost as cat

# ===== tensorflow =====
import tensorflow as tf
from tensorflow import random as tf_rnd
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers
from tensorflow.keras import metrics as tf_metrics
from tensorflow.keras import callbacks as tf_callbacks
from tqdm.keras import TqdmCallback
import tensorflow_addons as tfa
from tensorflow.keras.utils import plot_model
from keras.utils.layer_utils import count_params

# import keras_tuner as kt
# from keras_tuner import HyperModel
import tensorflow_hub as tf_hub
import tensorflow_recommenders as tfrs

# # # ===== pytorch =====
# import torch
# from torch.utils.data import DataLoader
# from transformers import AutoTokenizer
# from transformers import AutoModel

import librosa

# GPU memory setting
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    print(e)

warnings.filterwarnings(action='ignore')
rcParams['axes.unicode_minus'] = False
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 200)
# plt.rc('font', family='NanumSquareB')

path error : /usr/local/lib/python3.7/site-packages/missingpy/knnimpute.py


In [18]:
# ===== utility functions =====
# label encoding for categorical column with excepting na value
def seed_everything(seed=42):
    # python random module
    rnd.seed(seed)
    # numpy random
    np_rnd.seed(seed)
    # tf random
    try:
        tf_rnd.set_seed(seed)
    except:
        pass
    # RAPIDS random
    try:
        cp.random.seed(seed)
    except:
        pass
    # pytorch random
    try:
        torch.manual_seed(seed)
    except:
        pass
def which(bool_list):
    return where(bool_list)[0]
def easyIO(x=None, path=None, op="r"):
    tmp = None
    if op == "r":
        with open(path, "rb") as f:
            tmp = pickle.load(f)
        return tmp
    elif op == "w":
        with open(path, "wb") as f:
            pickle.dump(x, f)
    else:
        print("Unknown operation type")
def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]
def findIdx(data_x, col_names):
    return [int(i) for i, j in enumerate(data_x) if j in col_names]
def orderElems(for_order, using_ref):
    return [i for i in using_ref if i in for_order]
# concatenate by row
def cbr(df1, df2):
    if type(df1) == series:
        tmp_concat = series(pd.concat([dataframe(df1), dataframe(df2)], axis=0, ignore_index=True).iloc[:,0])
        tmp_concat.reset_index(drop=True, inplace=True)
    elif type(df1) == dataframe:
        tmp_concat = pd.concat([df1, df2], axis=0, ignore_index=True)
        tmp_concat.reset_index(drop=True, inplace=True)
    elif type(df1) == np.ndarray:
        tmp_concat = np.concatenate([df1, df2], axis=0)
    else:
        print("Unknown Type: return 1st argument")
        tmp_concat = df1
    return tmp_concat
def change_width(ax, new_value):
    for patch in ax.patches :
        current_width = patch.get_width()
        adj_value = current_width - new_value
        # we change the bar width
        patch.set_width(new_value)
        # we recenter the bar
        patch.set_x(patch.get_x() + adj_value * .5)
def week_of_month(date):
    month = date.month
    week = 0
    while date.month == month:
        week += 1
        date -= timedelta(days=7)
    return week
def getSeason(date):
    month = date.month
    if month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    elif month in [9, 10, 11]:
        return "Fall"
    else:
        return "Winter"
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error: Creating directory. ' + directory)
def sigmoid(x):
    return 1/(1 + np.exp(-x))
def dispPerformance(result_dic):
    perf_table = dataframe()
    index_names = []
    for k, v in result_dic.items():
        index_names.append(k)
        perf_table = pd.concat([perf_table, series(v["performance"]).to_frame().T], ignore_index=True, axis=0)
    perf_table.index = index_names
    perf_table.sort_values(perf_table.columns[0], inplace=True)
    print(perf_table)
    return perf_table
def powspace(start, stop, power, num):
    start = np.power(start, 1/float(power))
    stop = np.power(stop, 1/float(power))
    return np.power(np.linspace(start, stop, num=num), power)
def xgb_custom_lossfunction(alpha = 1):
    def support_under_mse(label, pred):
        # grad : 1차 미분
        # hess : 2차 미분
        residual = (label - pred).astype("float")
        grad = np.where(residual > 0, -2 * alpha * residual, -2 * residual)
        hess = np.where(residual > 0, 2 * alpha, 2.0)
        return grad, hess
    return support_under_mse
def pd_flatten(df):
    df = df.unstack()
    df.index = [str(i) + "_" + str(j) for i, j in df.index]
    return df
def tf_losses_rmse(y_true, y_pred, sample_weight=None):
    return tf.sqrt(tf.reduce_mean((y_true - y_pred) ** 2)) if sample_weight is None else tf.sqrt(tf.reduce_mean(((y_true - y_pred) ** 2) * sample_weight))
def tf_loss_nmae(y_true, y_pred, sample_weight=False):
    mae = tf.reduce_mean(tf.math.abs(y_true - y_pred))
    score = tf.math.divide(mae, tf.reduce_mean(tf.math.abs(y_true)))
    return score
def text_extractor(string, lang="eng", spacing=True):
    # # 괄호를 포함한 괄호 안 문자 제거 정규식
    # re.sub(r'\([^)]*\)', '', remove_text)
    # # <>를 포함한 <> 안 문자 제거 정규식
    # re.sub(r'\<[^)]*\>', '', remove_text)
    if lang == "eng":
        text_finder = re.compile('[^ A-Za-z]') if spacing else re.compile('[^A-Za-z]')
    elif lang == "kor":
        text_finder = re.compile('[^ ㄱ-ㅣ가-힣+]') if spacing else re.compile('[^ㄱ-ㅣ가-힣+]')
    # default : kor + eng
    else:
        text_finder = re.compile('[^ A-Za-zㄱ-ㅣ가-힣+]') if spacing else re.compile('[^A-Za-zㄱ-ㅣ가-힣+]')
    return text_finder.sub('', string)
def memory_usage(message='debug'):
    # current process RAM usage
    p = psutil.Process()
    rss = p.memory_info().rss / 2 ** 20 # Bytes to MB
    print(f"[{message}] memory usage: {rss: 10.3f} MB")
    return rss
def cos_sim(a, b):
  return dot(a, b)/(norm(a) * norm(b))
class MyLabelEncoder:
    def __init__(self, preset={}):
        # dic_cat format -> {"col_name": {"value": replace}}
        self.dic_cat = preset
    def fit_transform(self, data_x, col_names):
        tmp_x = copy.deepcopy(data_x)
        for i in col_names:
            # if key is not in dic, update dic
            if i not in self.dic_cat.keys():
                tmp_dic = dict.fromkeys(sorted(set(tmp_x[i]).difference([nan])))
                label_cnt = 0
                for j in tmp_dic.keys():
                    tmp_dic[j] = label_cnt
                    label_cnt += 1
                self.dic_cat[i] = tmp_dic
            # transform value which is not in dic to nan
            tmp_x[i] = tmp_x[i].astype("object")
            conv = tmp_x[i].replace(self.dic_cat[i])
            for conv_idx, j in enumerate(conv):
                if j not in self.dic_cat[i].values():
                    conv[conv_idx] = nan
            # final return
            tmp_x[i] = conv.astype("float")
        return tmp_x
    def transform(self, data_x):
        tmp_x = copy.deepcopy(data_x)
        for i in self.dic_cat.keys():
            # transform value which is not in dic to nan
            tmp_x[i] = tmp_x[i].astype("object")
            conv = tmp_x[i].replace(self.dic_cat[i])
            for conv_idx, j in enumerate(conv):
                if j not in self.dic_cat[i].values():
                    conv[conv_idx] = nan
            # final return
            tmp_x[i] = conv.astype("float")
        return tmp_x
    def clear(self):
        self.dic_cat = {}
class MyOneHotEncoder:
    def __init__(self, label_preset={}):
        self.dic_cat = {}
        self.label_preset = label_preset
    def fit_transform(self, data_x, col_names):
        tmp_x = dataframe()
        for i in data_x:
            if i not in col_names:
                tmp_x = pd.concat([tmp_x, dataframe(data_x[i])], axis=1)
            else:
                if not ((data_x[i].dtype.name == "object") or (data_x[i].dtype.name == "category")):
                    print(F"WARNING : {i} is not object or category")
                self.dic_cat[i] = OneHotEncoder(sparse=False, handle_unknown="ignore")
                conv = self.dic_cat[i].fit_transform(dataframe(data_x[i])).astype("int")
                col_list = []
                for j in self.dic_cat[i].categories_[0]:
                    if i in self.label_preset.keys():
                        for k, v in self.label_preset[i].items():
                            if v == j:
                                col_list.append(str(i) + "_" + str(k))
                    else:
                        col_list.append(str(i) + "_" + str(j))
                conv = dataframe(conv, columns=col_list)
                tmp_x = pd.concat([tmp_x, conv], axis=1)
        return tmp_x
    def transform(self, data_x):
        tmp_x = dataframe()
        for i in data_x:
            if not i in list(self.dic_cat.keys()):
                tmp_x = pd.concat([tmp_x, dataframe(data_x[i])], axis=1)
            else:
                if not ((data_x[i].dtype.name == "object") or (data_x[i].dtype.name == "category")):
                    print(F"WARNING : {i} is not object or category")
                conv = self.dic_cat[i].transform(dataframe(data_x[i])).astype("int")
                col_list = []
                for j in self.dic_cat[i].categories_[0]:
                    if i in self.label_preset.keys():
                        for k, v in self.label_preset[i].items():
                            if v == j: col_list.append(str(i) + "_" + str(k))
                    else:
                        col_list.append(str(i) + "_" + str(j))
                conv = dataframe(conv, columns=col_list)
                tmp_x = pd.concat([tmp_x, conv], axis=1)
        return tmp_x
    def clear(self):
        self.dic_cat = {}
        self.label_preset = {}
class MyKNNImputer:
    def __init__(self, k=5):
        self.imputer = KNNImputer(n_neighbors=k)
        self.dic_cat = {}
    def fit_transform(self, x, cat_vars=None):
        if cat_vars is None:
            x_imp = dataframe(self.imputer.fit_transform(x), columns=x.columns)
        else:
            naIdx = dict.fromkeys(cat_vars)
            for i in cat_vars:
                self.dic_cat[i] = diff(list(sorted(set(x[i]))), [nan])
                naIdx[i] = list(which(array(x[i].isna())))
            x_imp = dataframe(self.imputer.fit_transform(x), columns=x.columns)

            # if imputed categorical value are not in the range, adjust the value
            for i in cat_vars:
                x_imp[i] = x_imp[i].apply(lambda x: int(round(x, 0)))
                for j in naIdx[i]:
                    if x_imp[i][j] not in self.dic_cat[i]:
                        if x_imp[i][j] < self.dic_cat[i][0]:
                            x_imp[i][naIdx[i]] = self.dic_cat[i][0]
                        elif x_imp[i][j] > self.dic_cat[i][0]:
                            x_imp[i][naIdx[i]] = self.dic_cat[i][len(self.dic_cat[i]) - 1]
        return x_imp
    def transform(self, x):
        if len(self.dic_cat.keys()) == 0:
            x_imp = dataframe(self.imputer.transform(x), columns=x.columns)
        else:
            naIdx = dict.fromkeys(self.dic_cat.keys())
            for i in self.dic_cat.keys():
                naIdx[i] = list(which(array(x[i].isna())))
            x_imp = dataframe(self.imputer.transform(x), columns=x.columns)

            # if imputed categorical value are not in the range, adjust the value
            for i in self.dic_cat.keys():
                x_imp[i] = x_imp[i].apply(lambda x: int(round(x, 0)))
                for j in naIdx[i]:
                    if x_imp[i][j] not in self.dic_cat[i]:
                        if x_imp[i][j] < self.dic_cat[i][0]:
                            x_imp[i][naIdx[i]] = self.dic_cat[i][0]
                        elif x_imp[i][j] > self.dic_cat[i][0]:
                            x_imp[i][naIdx[i]] = self.dic_cat[i][len(self.dic_cat[i]) - 1]
        return x_imp
    def clear(self):
        self.imputer = None
        self.dic_cat = {}
def remove_outlier(df, std=3, mode="remove"):
    tmp_df = df.copy()
    if mode == "remove":
        outlier_mask = (np.abs(stats.zscore(tmp_df)) > std).all(axis=1)
        print("found outlier :", outlier_mask.sum())
        tmp_df = tmp_df[~outlier_mask]
    elif mode == "interpolate":
        tmp_outlier = []
        for i in tmp_df:
            outlier_mask = (np.abs(stats.zscore(tmp_df[i])) > std)
            tmp_outlier.append(outlier_mask.sum())
            if tmp_outlier[-1] == 0:
                continue
            tmp_df[i][outlier_mask] = np.nan
            tmp_df[i] = tmp_df[i].interpolate(method='linear').bfill()
        print("found outlier :", np.sum(outlier_mask))
    return tmp_df
def convert_sparse_matrix_to_sparse_tensor(X, sorted=True):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.sparse.reorder(tf.SparseTensor(indices, coo.data, coo.shape)) if sorted else tf.SparseTensor(indices, coo.data, coo.shape)
seed_everything()

In [19]:
folder_path = "/content/drive/MyDrive/Colab Notebooks/projects/Dacon/covid19_diagnostics/"

# Preprocessing

In [20]:
df_full = pd.read_csv(folder_path + 'open.zip (Unzipped Files)/train_data.csv')
df_test = pd.read_csv(folder_path + 'open.zip (Unzipped Files)/test_data.csv')

In [21]:
df_full.head()

,id,age,gender,respiratory_condition,fever_or_muscle_pain,covid19
0,1,24,female,0,1,0
1,2,51,male,0,0,0
2,3,22,male,0,0,0
3,4,29,female,1,0,0
4,5,23,male,0,0,0


In [22]:
df_test.head()

,id,age,gender,respiratory_condition,fever_or_muscle_pain
0,3806,48,female,1,0
1,3807,24,female,0,0
2,3808,29,male,0,0
3,3809,39,female,0,0
4,3810,34,male,0,0


In [23]:
def get_mfcc_feature(df, data_type):
    # Data Folder path
    root_folder = folder_path + 'open.zip (Unzipped Files)/'
    features = []
    for uid in tqdm(df['id']):
        root_path = os.path.join(root_folder, data_type)
        path = os.path.join(root_path, str(uid).zfill(5)+'.wav')

        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=CFG['SR'])
        
        # librosa패키지를 사용하여 mfcc 추출
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CFG['N_MFCC'])

        y_feature = []
        # 추출된 MFCC들의 평균을 Feature로 사용
        for e in mfcc:
            y_feature.append(np.mean(e))
        features.append(y_feature)
    
    # 기존의 자가진단 정보를 담은 데이터프레임에 추출된 오디오 Feature를 추가
    mfcc_df = pd.DataFrame(features, columns=['mfcc_'+str(x) for x in range(1,CFG['N_MFCC']+1)])
    df = pd.concat([df, mfcc_df], axis=1)
    return df
    print('Done.')

In [24]:
def get_vggish_feature(df, data_type):
    pretrained_vggish = tf_hub.load('https://tfhub.dev/google/vggish/1')
    avg_pooling = layers.GlobalAveragePooling1D()

    # Data Folder path
    root_folder = folder_path + 'open.zip (Unzipped Files)/'
    features = []
    for uid in tqdm(df['id']):
        root_path = os.path.join(root_folder, data_type)
        path = os.path.join(root_path, str(uid).zfill(5)+'.wav')

        # librosa패키지를 사용하여 wav 파일 load
        # sampling rate : 16000 (16kHz)
        y, sr = librosa.load(path, sr=16000)

        # getting averaged vggish feature
        y = pretrained_vggish(y)
        features.append(avg_pooling(tf.expand_dims(y, 0)).numpy()[0])
    
    # 기존의 자가진단 정보를 담은 데이터프레임에 추출된 오디오 Feature를 추가
    feature_df = pd.DataFrame(features, columns=['vggish_' + str(x) for x in range(128)])
    if feature_df.isna().sum().sum() > 0:
        print("INFO : df includes na values")
    df = pd.concat([df, feature_df], axis=1)
    return df
    print('Done.')

In [26]:
CFG = {
    'SR':16000,
    'N_MFCC':64, # MFCC 벡터를 추출할 개수
    'SEED':41
}

df_full = get_mfcc_feature(df_full, 'train')
df_test = get_mfcc_feature(df_test, 'test')

100%|██████████| 5732/5732 [1:32:44<00:00,  1.03it/s]


In [27]:
easyIO(df_full, folder_path + "dataset/train_mfcc_64.csv", "w")
easyIO(df_test, folder_path + "dataset/test_mfcc_64.csv", "w")

In [ ]:
# df_full = get_mfcc_feature(df_full, 'train')
# df_test = get_mfcc_feature(df_test, 'test')

100%|██████████| 5732/5732 [1:59:49<00:00,  1.25s/it]


In [ ]:
CFG = {
    'SR':16000,
    'N_MFCC':32, # MFCC 벡터를 추출할 개수
    'SEED':41
}

# easyIO(df_full, folder_path + "dataset/train_mfcc.csv", "w")
# easyIO(df_test, folder_path + "dataset/test_mfcc.csv", "w")

In [ ]:
# df_full = get_vggish_feature(df_full, 'train')
# df_test = get_vggish_feature(df_test, 'test')

100%|██████████| 5732/5732 [1:41:12<00:00,  1.06s/it]


INFO : df includes na values


In [ ]:
# easyIO(df_full, folder_path + "dataset/train_vggish.csv", "w")
# easyIO(df_test, folder_path + "dataset/test_vggish.csv", "w")

# Quick Start Session

In [ ]:
# audio_feature_type = "mfcc"
audio_feature_type = "vggish"

In [ ]:
target_var = "covid19"
num_vars = ["age"]
bin_vars = ["gender", "respiratory_condition", "fever_or_muscle_pain"]
cat_vars = []

if audio_feature_type == "vggish":
    audio_vars = [audio_feature_type + "_" + str(i) for i in range(128)]
elif audio_feature_type == "mfcc":
    audio_vars = [audio_feature_type + "_" + str(i) for i in range(1,CFG['N_MFCC']+1)]

In [ ]:
df_full = easyIO(None, folder_path + "dataset/train_" + audio_feature_type + ".csv", "r")
df_test = easyIO(None, folder_path + "dataset/test_" + audio_feature_type + ".csv", "r")

In [ ]:
df_full.drop("id", axis=1, inplace=True)
df_test.drop("id", axis=1, inplace=True)

In [ ]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3805 entries, 0 to 3804
Columns: 133 entries, age to vggish_127
dtypes: float32(128), int64(4), object(1)
memory usage: 2.0+ MB


In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5732 entries, 0 to 5731
Columns: 132 entries, age to vggish_127
dtypes: float32(128), int64(3), object(1)
memory usage: 3.0+ MB


# Feature Enginnering

In [ ]:
# for i in df_full:
#     print(df_full[i].value_counts(), "\n")

In [ ]:
tmp = []
for i in df_full["gender"]:
    if i == "male":
        tmp.append(1.0)
    elif i == "female":
        tmp.append(0.0)
    else:
        tmp.append(nan)
df_full["gender"] = tmp
df_full = df_full.dropna().reset_index(drop=True)

tmp = []
for i in df_test["gender"]:
    if i == "male":
        tmp.append(1.0)
    elif i == "female":
        tmp.append(0.0)
    else:
        tmp.append(0.5)
df_test["gender"] = tmp

In [ ]:
# age discretization for 8, 4 groups

df_full["age_disc_8"] = pd.cut(df_full["age"], bins=[-np.inf, 10, 20, 30, 40, 50, 60, 70, np.inf], right=False, labels=list(range(8))).astype("float32")
df_full["age_disc_4"] = pd.cut(df_full["age"], bins=[-np.inf, 20, 40, 60, np.inf], right=False, labels=list(range(4))).astype("float32")

df_test["age_disc_8"] = pd.cut(df_test["age"], bins=[-np.inf, 10, 20, 30, 40, 50, 60, 70, np.inf], right=False, labels=list(range(8))).astype("float32")
df_test["age_disc_4"] = pd.cut(df_test["age"], bins=[-np.inf, 20, 40, 60, np.inf], right=False, labels=list(range(4))).astype("float32")

df_full = pd.concat([df_full.drop("age_disc_8", axis=1), dataframe(tf.keras.utils.to_categorical(df_full["age_disc_8"], num_classes=8, dtype='float32'), columns=["age_disc8_" + str(i) for i in range(8)])], axis=1)
df_full = pd.concat([df_full.drop("age_disc_4", axis=1), dataframe(tf.keras.utils.to_categorical(df_full["age_disc_4"], num_classes=4, dtype='float32'), columns=["age_disc4_" + str(i) for i in range(4)])], axis=1)

df_test = pd.concat([df_test.drop("age_disc_8", axis=1), dataframe(tf.keras.utils.to_categorical(df_test["age_disc_8"], num_classes=8, dtype='float32'), columns=["age_disc8_" + str(i) for i in range(8)])], axis=1)
df_test = pd.concat([df_test.drop("age_disc_4", axis=1), dataframe(tf.keras.utils.to_categorical(df_test["age_disc_4"], num_classes=4, dtype='float32'), columns=["age_disc4_" + str(i) for i in range(4)])], axis=1)

In [ ]:
# covid19 symptom interaction (2:both, 1:either, 0:neither)

df_full["symptom_class"] = (df_full["respiratory_condition"] + df_full["fever_or_muscle_pain"]).astype("float32")
df_test["symptom_class"] = (df_test["respiratory_condition"] + df_test["fever_or_muscle_pain"]).astype("float32")

df_full = pd.concat([df_full.drop("symptom_class", axis=1), dataframe(tf.keras.utils.to_categorical(df_full["symptom_class"], num_classes=3, dtype='float32'), columns=["symptom_class_" + str(i) for i in range(3)])], axis=1)
df_test = pd.concat([df_test.drop("symptom_class", axis=1), dataframe(tf.keras.utils.to_categorical(df_test["symptom_class"], num_classes=3, dtype='float32'), columns=["symptom_class_" + str(i) for i in range(3)])], axis=1)

In [ ]:
# vulnerable class (age>=60 and symptom_class==2)

df_full["vulnerable"] = (df_full["symptom_class_2"].astype("bool") & df_full["age_disc4_3"].astype("bool")).astype("float32")
df_test["vulnerable"] = (df_test["symptom_class_2"].astype("bool") & df_test["age_disc4_3"].astype("bool")).astype("float32")

bin_vars += ["vulnerable"]

In [ ]:
num_vars = diff(df_full.columns, bin_vars + cat_vars + audio_vars + [target_var])

In [ ]:
df_full[num_vars + bin_vars + audio_vars] = df_full[num_vars + bin_vars + audio_vars].astype("float32")
df_full[target_var] = df_full[target_var].astype("int32")

df_test[num_vars + bin_vars + audio_vars] = df_test[num_vars + bin_vars + audio_vars].astype("float32")

In [ ]:
num_vars

['age',
 'age_disc8_0',
 'age_disc8_1',
 'age_disc8_2',
 'age_disc8_3',
 'age_disc8_4',
 'age_disc8_5',
 'age_disc8_6',
 'age_disc8_7',
 'age_disc4_0',
 'age_disc4_1',
 'age_disc4_2',
 'age_disc4_3',
 'symptom_class_0',
 'symptom_class_1',
 'symptom_class_2']

In [ ]:
bin_vars

['gender', 'respiratory_condition', 'fever_or_muscle_pain', 'vulnerable']

In [ ]:
bin_vars += num_vars[1:]
num_vars = ["age"]

In [ ]:
bin_vars

['gender',
 'respiratory_condition',
 'fever_or_muscle_pain',
 'vulnerable',
 'age_disc8_0',
 'age_disc8_1',
 'age_disc8_2',
 'age_disc8_3',
 'age_disc8_4',
 'age_disc8_5',
 'age_disc8_6',
 'age_disc8_7',
 'age_disc4_0',
 'age_disc4_1',
 'age_disc4_2',
 'age_disc4_3',
 'symptom_class_0',
 'symptom_class_1',
 'symptom_class_2']

In [ ]:
cat_vars

[]

In [ ]:
audio_vars

['vggish_0',
 'vggish_1',
 'vggish_2',
 'vggish_3',
 'vggish_4',
 'vggish_5',
 'vggish_6',
 'vggish_7',
 'vggish_8',
 'vggish_9',
 'vggish_10',
 'vggish_11',
 'vggish_12',
 'vggish_13',
 'vggish_14',
 'vggish_15',
 'vggish_16',
 'vggish_17',
 'vggish_18',
 'vggish_19',
 'vggish_20',
 'vggish_21',
 'vggish_22',
 'vggish_23',
 'vggish_24',
 'vggish_25',
 'vggish_26',
 'vggish_27',
 'vggish_28',
 'vggish_29',
 'vggish_30',
 'vggish_31',
 'vggish_32',
 'vggish_33',
 'vggish_34',
 'vggish_35',
 'vggish_36',
 'vggish_37',
 'vggish_38',
 'vggish_39',
 'vggish_40',
 'vggish_41',
 'vggish_42',
 'vggish_43',
 'vggish_44',
 'vggish_45',
 'vggish_46',
 'vggish_47',
 'vggish_48',
 'vggish_49',
 'vggish_50',
 'vggish_51',
 'vggish_52',
 'vggish_53',
 'vggish_54',
 'vggish_55',
 'vggish_56',
 'vggish_57',
 'vggish_58',
 'vggish_59',
 'vggish_60',
 'vggish_61',
 'vggish_62',
 'vggish_63',
 'vggish_64',
 'vggish_65',
 'vggish_66',
 'vggish_67',
 'vggish_68',
 'vggish_69',
 'vggish_70',
 'vggish_71',
 '

In [ ]:
df_full.head()

,age,gender,respiratory_condition,fever_or_muscle_pain,covid19,vggish_0,vggish_1,vggish_2,vggish_3,vggish_4,vggish_5,vggish_6,vggish_7,vggish_8,vggish_9,vggish_10,vggish_11,vggish_12,vggish_13,vggish_14,vggish_15,vggish_16,vggish_17,vggish_18,vggish_19,vggish_20,vggish_21,vggish_22,vggish_23,vggish_24,vggish_25,vggish_26,vggish_27,vggish_28,vggish_29,vggish_30,vggish_31,vggish_32,vggish_33,vggish_34,vggish_35,vggish_36,vggish_37,vggish_38,vggish_39,vggish_40,vggish_41,vggish_42,vggish_43,vggish_44,...,vggish_94,vggish_95,vggish_96,vggish_97,vggish_98,vggish_99,vggish_100,vggish_101,vggish_102,vggish_103,vggish_104,vggish_105,vggish_106,vggish_107,vggish_108,vggish_109,vggish_110,vggish_111,vggish_112,vggish_113,vggish_114,vggish_115,vggish_116,vggish_117,vggish_118,vggish_119,vggish_120,vggish_121,vggish_122,vggish_123,vggish_124,vggish_125,vggish_126,vggish_127,age_disc8_0,age_disc8_1,age_disc8_2,age_disc8_3,age_disc8_4,age_disc8_5,age_disc8_6,age_disc8_7,age_disc4_0,age_disc4_1,age_disc4_2,age_disc4_3,symptom_class_0,symptom_class_1,symptom_class_2,vulnerable
0,24.0,0.0,0.0,1.0,0,0.074367,-0.304656,0.444292,-0.578650,-0.150864,-0.474929,-0.407196,-0.495602,-0.878000,0.271428,-0.539890,-0.369853,-0.417934,-0.303931,-0.110700,-0.402886,-0.192091,0.479554,-0.333110,-0.190150,0.298838,-0.561066,-0.413499,-0.085026,0.340623,0.327000,0.929875,-0.025833,-0.176998,-0.227616,-0.287722,-0.334935,0.053041,0.741705,0.431359,-0.110280,-0.567165,0.028924,-0.363699,-0.180546,0.569059,-0.235530,0.069347,-0.232013,0.163835,...,0.030110,0.276897,-0.025295,-0.758403,-0.413647,0.010866,-0.096699,-0.015649,0.273723,0.238724,-0.394732,-0.303550,0.195753,-0.705073,0.280866,-0.297937,0.119591,-0.420765,-0.227821,-0.245124,-0.604481,-0.158429,-0.603614,-0.202446,-0.877170,0.174617,0.055384,-0.305561,-0.340612,0.682212,0.114898,-0.233344,-0.286952,-0.310891,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,51.0,1.0,0.0,0.0,0,-0.093080,-0.323049,0.228054,-0.253214,-0.201783,-0.593411,-0.281786,-0.253731,-0.278699,-0.072951,-0.490047,-0.503245,-0.350297,-0.339051,-0.002062,-0.231233,-0.181500,0.311492,-0.262562,-0.192817,0.545472,-0.518024,-0.137717,0.070471,0.228461,0.560039,0.633050,0.086444,-0.147594,-0.133464,-0.160392,-0.177724,0.185062,0.741338,0.196328,-0.076370,-0.549249,-0.055279,-0.254215,-0.142908,0.512294,-0.259097,0.063067,-0.161039,0.093234,...,0.027641,0.237238,-0.089823,-0.546613,-0.461019,-0.062111,0.377900,0.031675,-0.014577,0.253654,-0.366609,-0.412507,0.349866,-0.466986,0.318550,-0.399750,0.162004,-0.444358,-0.055705,-0.014060,-0.342550,-0.254980,-0.435774,0.006863,-0.590584,-0.026447,0.032024,-0.168996,-0.115205,0.099536,-0.089518,-0.368148,-0.261253,-0.418885,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,22.0,1.0,0.0,0.0,0,-0.164700,-0.172472,0.426343,-0.175452,0.010170,-0.466768,-0.258091,-0.289365,-0.301333,-0.068874,-0.358127,-0.365384,-0.336614,-0.281883,-0.151231,-0.172183,0.013820,0.290865,-0.238683,-0.257594,0.262282,-0.429282,-0.177642,0.039664,0.273470,0.383673,0.420238,0.081119,0.034558,-0.084735,-0.173926,-0.308173,0.111967,0.845682,0.424526,-0.100941,-0.512767,-0.113160,-0.115093,-0.165239,0.395894,-0.116302,0.221822,-0.130291,0.172963,...,-0.022540,0.287411,-0.111606,-0.499588,-0.290404,-0.099194,0.083804,-0.012489,0.108274,0.191909,-0.301285,-0.486360,0.326836,-0.340729,0.354032,-0.345735,0.107432,-0.253056,-0.178793,0.008702,-0.317845,0.021927,-0.378676,-0.128969,-0.657660,-0.003223,0.178623,-0.175769,-0.100840,0.230077,-0.014840,-0.122037,-0.041008,-0.299935,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,29.0,0.0,1.0,0.0,0,0.088070,-0.352619,0.365449,-0.385953,-0.079941,-0.321737,-0.384355,-0.383507,-0.441021,0.317373,-0.634525,-0.614604,-0.300173,-0.422857,-0.180514,-0.445056,-0.255017,0.234226,-0.319845,-0.330160,0.205797,-0.450726,-0.244148,0.105189,0.334995,0.458771,0.730664,-0.042996,-0.139413,-0.277734,-0.203148,-0.286275,0.150699,0.536051,0.327327,-0.108

In [ ]:
df_test.head()

,age,gender,respiratory_condition,fever_or_muscle_pain,vggish_0,vggish_1,vggish_2,vggish_3,vggish_4,vggish_5,vggish_6,vggish_7,vggish_8,vggish_9,vggish_10,vggish_11,vggish_12,vggish_13,vggish_14,vggish_15,vggish_16,vggish_17,vggish_18,vggish_19,vggish_20,vggish_21,vggish_22,vggish_23,vggish_24,vggish_25,vggish_26,vggish_27,vggish_28,vggish_29,vggish_30,vggish_31,vggish_32,vggish_33,vggish_34,vggish_35,vggish_36,vggish_37,vggish_38,vggish_39,vggish_40,vggish_41,vggish_42,vggish_43,vggish_44,vggish_45,...,vggish_94,vggish_95,vggish_96,vggish_97,vggish_98,vggish_99,vggish_100,vggish_101,vggish_102,vggish_103,vggish_104,vggish_105,vggish_106,vggish_107,vggish_108,vggish_109,vggish_110,vggish_111,vggish_112,vggish_113,vggish_114,vggish_115,vggish_116,vggish_117,vggish_118,vggish_119,vggish_120,vggish_121,vggish_122,vggish_123,vggish_124,vggish_125,vggish_126,vggish_127,age_disc8_0,age_disc8_1,age_disc8_2,age_disc8_3,age_disc8_4,age_disc8_5,age_disc8_6,age_disc8_7,age_disc4_0,age_disc4_1,age_disc4_2,age_disc4_3,symptom_class_0,symptom_class_1,symptom_class_2,vulnerable
0,48.0,0.0,1.0,0.0,-0.249435,-0.078036,-0.000848,0.146939,-0.045798,-0.159696,-0.107600,-0.118128,-0.229618,-0.085126,-0.235071,-0.259579,0.385549,0.086088,-0.055775,-0.087479,0.143160,0.361288,0.108084,-0.330335,-0.034617,-0.221370,-0.095338,-0.120964,0.247845,0.081271,0.015907,0.196405,0.307760,-0.038643,-0.072448,-0.123231,0.097034,0.271419,0.324465,-0.217953,-0.258507,-0.049462,-0.064540,-0.146689,0.439659,-0.045267,0.085254,-0.078951,0.541457,-0.115800,...,0.206394,0.211534,-0.039425,-0.022362,-0.067527,0.073929,0.098903,0.455746,0.028536,0.340663,-0.370271,-0.114770,0.183220,-0.307789,0.077042,-0.072964,-0.037340,0.489214,0.029793,-0.089756,-0.226815,0.011774,-0.221705,0.017363,-0.237012,-0.021080,-0.035114,-0.075009,-0.027227,-0.024461,0.132206,-0.164106,0.161094,0.090653,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,24.0,0.0,0.0,0.0,-0.365054,-0.158612,0.104900,0.102022,-0.092483,-0.316609,-0.133178,-0.212336,-0.312242,-0.101015,-0.244832,-0.253776,0.163712,-0.070663,-0.103565,-0.139592,0.103058,0.406807,0.004901,-0.351940,0.073389,-0.343818,-0.119598,-0.132139,0.204288,0.271953,0.231882,0.171854,0.296670,-0.113120,-0.056802,-0.177380,0.068660,0.481462,0.246493,-0.197000,-0.326874,-0.126799,-0.089023,-0.181524,0.418555,-0.128327,0.135198,-0.115831,0.394721,-0.059560,...,0.181847,0.237135,-0.081053,-0.158384,-0.192924,-0.014655,0.253809,0.304555,0.006955,0.353097,-0.396016,-0.249069,0.200262,-0.315313,0.082205,-0.155889,0.002312,0.223830,-0.033760,-0.098017,-0.280393,0.009500,-0.320952,0.001696,-0.343810,-0.123523,-0.024368,-0.177301,-0.066536,-0.022950,0.021847,-0.326030,0.092071,-0.040008,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,29.0,1.0,0.0,0.0,-0.266173,-0.051474,-0.135684,0.300016,-0.048960,-0.072094,-0.043378,-0.105313,-0.176645,-0.053787,-0.157787,-0.235288,0.587219,0.159131,-0.093491,-0.038367,0.234486,0.322382,0.232740,-0.354743,-0.063508,-0.165087,-0.044461,-0.133523,0.226273,-0.025890,-0.119254,0.260348,0.438883,-0.014112,-0.039560,-0.134782,0.144375,0.078759,0.245981,-0.261044,-0.198053,-0.057812,-0.076063,-0.125583,0.376237,-0.013202,0.051288,-0.074818,0.592029,-0.133659,...,0.244246,0.253046,-0.031076,0.113678,-0.049720,0.099078,0.119885,0.541312,-0.055865,0.269268,-0.384827,-0.104771,0.138724,-0.228172,0.007556,-0.020042,-0.065018,0.687625,0.074215,-0.045640,-0.177862,0.054304,-0.169483,0.023931,-0.093769,-0.032683,-0.112096,-0.030724,-0.032954,-0.105866,0.207592,-0.156844,0.227877,0.175801,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,39.0,0.0,0.0,0.0,0.081596,-0.050033,0.091697,-0.195561,-0.050593,-0.308308,-0.194966,-0.248588,-0.258360,0.204068,-0.289227,-0.391996,-0.264257,-0.129608,-0.087989,-0.159570,-0.054177,0.388775,-0.044596,-0.044411,0.229416,-0.203256,-0.181956,-0.072175,0.544561,0.309737,0.507254,0.135759,0.069490,-0.173852,-0.133196,-0.101946,-0.019489,0.627335,0.28204

In [ ]:
print(df_full.isna().sum().sum())
print(df_test.isna().sum().sum())

0
896


In [ ]:
df_test.isna().sum().index[df_test.isna().sum() > 0]

Index(['vggish_0', 'vggish_1', 'vggish_2', 'vggish_3', 'vggish_4', 'vggish_5', 'vggish_6', 'vggish_7', 'vggish_8', 'vggish_9',
       ...
       'vggish_118', 'vggish_119', 'vggish_120', 'vggish_121', 'vggish_122', 'vggish_123', 'vggish_124', 'vggish_125', 'vggish_126', 'vggish_127'], dtype='object', length=128)

In [ ]:
# linear_model,
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

# make_pipeline(StandardScaler(), LinearRegression())

imputor = IterativeImputer(
    estimator=make_pipeline(StandardScaler(), LinearRegression()),
    max_iter=10, random_state=42
)

# imputing na values on audio features in test dataset
imputor.fit(df_full.drop(target_var, axis=1).to_numpy())

df_test.iloc[:,:] = imputor.transform(df_test.to_numpy())

In [ ]:
# # imputing na values on audio features in test dataset

# imputor = MissForest(max_iter=10, n_estimators=100, max_depth=4, min_samples_leaf=3, verbose=1, random_state=42)
# imputor.fit(df_full.drop(target_var, axis=1).to_numpy())

# df_test[:,:] = imputor.transform(df_test.to_numpy())

In [ ]:
print(df_full.isna().sum().sum())
print(df_test.isna().sum().sum())

0
0


In [ ]:
df_full = df_full.drop_duplicates().reset_index(drop=True)
df_full.sample(frac=1, random_state=42).reset_index(drop=True)

,age,gender,respiratory_condition,fever_or_muscle_pain,covid19,vggish_0,vggish_1,vggish_2,vggish_3,vggish_4,vggish_5,vggish_6,vggish_7,vggish_8,vggish_9,vggish_10,vggish_11,vggish_12,vggish_13,vggish_14,vggish_15,vggish_16,vggish_17,vggish_18,vggish_19,vggish_20,vggish_21,vggish_22,vggish_23,vggish_24,vggish_25,vggish_26,vggish_27,vggish_28,vggish_29,vggish_30,vggish_31,vggish_32,vggish_33,vggish_34,vggish_35,vggish_36,vggish_37,vggish_38,vggish_39,vggish_40,vggish_41,vggish_42,vggish_43,vggish_44,...,vggish_94,vggish_95,vggish_96,vggish_97,vggish_98,vggish_99,vggish_100,vggish_101,vggish_102,vggish_103,vggish_104,vggish_105,vggish_106,vggish_107,vggish_108,vggish_109,vggish_110,vggish_111,vggish_112,vggish_113,vggish_114,vggish_115,vggish_116,vggish_117,vggish_118,vggish_119,vggish_120,vggish_121,vggish_122,vggish_123,vggish_124,vggish_125,vggish_126,vggish_127,age_disc8_0,age_disc8_1,age_disc8_2,age_disc8_3,age_disc8_4,age_disc8_5,age_disc8_6,age_disc8_7,age_disc4_0,age_disc4_1,age_disc4_2,age_disc4_3,symptom_class_0,symptom_class_1,symptom_class_2,vulnerable
0,27.0,1.0,0.0,0.0,0,0.237233,-0.073406,0.142055,-0.539462,-0.054550,-0.457476,-0.277106,-0.374962,-0.193999,0.135108,-0.685005,-0.552347,-0.382332,-0.179146,0.085340,-0.287859,-0.125214,0.629780,-0.346372,-0.098754,0.320120,-0.379453,-0.325220,-0.036740,0.455812,0.213303,0.531806,0.213881,-0.179417,-0.270110,-0.239606,-0.155629,-0.018713,0.639012,0.427209,-0.128684,-0.552983,-0.000528,-0.309293,-0.021275,0.377659,-0.134463,0.198480,-0.261788,0.014866,...,0.097233,0.221708,0.016513,-0.646305,-0.445976,0.163898,-0.216428,-0.002031,0.221547,0.330069,-0.362868,-0.170088,0.376592,-0.461542,0.362308,-0.227808,-0.194761,-0.259064,-0.121842,0.059296,-0.381911,-0.278796,-0.497189,0.052893,-0.858647,0.253602,0.176621,-0.052145,-0.088078,0.347476,0.031137,-0.037026,-0.059619,-0.262939,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,26.0,0.0,0.0,0.0,0,0.246134,-0.148192,0.244273,-0.538205,-0.337523,-0.593265,-0.311994,-0.429375,-0.532566,0.282096,-0.808344,-0.636284,-0.525758,-0.101947,0.089822,-0.137083,-0.029002,0.338129,-0.202859,-0.015267,0.312917,-0.102637,-0.262040,0.089277,0.297634,0.217412,0.472763,0.407411,-0.235386,-0.319075,0.004311,-0.177443,0.011335,0.692153,0.180619,0.222378,-0.572982,-0.159462,-0.438346,0.069036,0.315911,0.065443,0.149918,-0.115082,0.204928,...,-0.058818,0.199952,-0.109857,-0.661890,-0.535261,-0.008445,-0.197057,0.303818,-0.003170,0.225542,-0.452083,-0.424431,0.354409,-0.191299,0.330623,-0.153216,-0.113831,-0.202335,-0.015565,-0.017402,-0.653620,-0.330262,-0.408605,0.028116,-1.047375,0.246120,0.070484,-0.083004,-0.088040,0.252036,-0.007256,0.090935,-0.067936,-0.358746,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,27.0,1.0,0.0,0.0,0,-0.061355,-0.104770,0.196464,0.256682,0.008873,-0.406786,-0.341376,-0.238717,-0.307947,0.039246,-0.475410,-0.464350,-0.124527,-0.061451,-0.096154,-0.199233,0.069042,0.077799,0.014512,-0.261682,0.219502,-0.286834,-0.178618,0.106887,0.232501,0.102398,0.084113,0.075658,0.324012,-0.062375,-0.094696,-0.273715,0.143767,0.459655,0.268433,-0.109267,-0.437196,0.252963,-0.247513,-0.107284,0.291054,-0.064484,0.082671,-0.155126,0.336953,...,0.224578,0.291128,0.063938,-0.272569,-0.237854,-0.038205,0.077042,0.109512,0.081803,0.132673,-0.329243,-0.353951,0.336664,-0.255079,0.467882,-0.175494,-0.225324,0.365005,0.024953,0.074411,-0.251430,0.023905,-0.355071,-0.024995,-0.566465,-0.008698,-0.067261,-0.180592,-0.229741,0.200737,0.144086,-0.227922,-0.003502,-0.203552,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,37.0,1.0,0.0,0.0,0,-0.134299,-0.150072,0.405094,-0.510863,-0.086288,-0.418875,0.001268,-0.251929,-0.498573,0.072656,-0.201595,-0.357154,-0.282462,-0.169092,-0.099354,-0.152573,-0.107116,0.530403,-0.247494,-0.064046,0.335060,-0.395883,-0.224059,-0.209563,0.361936,0.483415,0.785008,0.148584,-0.128865,-0.204075,-0.192620,-0.088760,-0.030923,0.896775,0.330546,-0.102979,-0.244311,-

# Define utility funcions

In [ ]:
# optuna function
def optuna_objective_function(trial: Trial, fold, train_x, train_y, train_groups, val_x, val_y, val_groups, categoIdx,
                              model_name, output_container, ntrees=1000, eta=1e-2):
    if model_name == "LGB_GOSS":
        tuning_params = {
            "num_leaves": trial.suggest_categorical("num_leaves", [pow(2, i) - 1 for i in [4, 5, 6, 7, 8]]),
            "subsample": trial.suggest_float("subsample", 0.5, 0.8, step=0.05),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 0.8, step=0.05),
            "reg_lambda": trial.suggest_float("reg_lambda", 0.001, 10.0, log=True),
            "min_child_weight": trial.suggest_float("min_child_weight", 0.001, 10.0, log=True),
            "min_child_samples": trial.suggest_int("min_child_samples", 3, 99, log=True),
            "scale_pos_weight": trial.suggest_float("scale_pos_weight", 0.2, 5.0, log=True),
        }
        cb_list = [
            lgb.early_stopping(stopping_rounds=int(ntrees * 0.2), first_metric_only=True, verbose=False, min_delta=0.001),
        ]

        model = lgb.LGBMClassifier(boosting_type="goss", objective="binary",
                                   n_estimators=ntrees, learning_rate=eta,
                                   random_state=fold, device_type="gpu",
                                   verbose=-1, **tuning_params)
        model.fit(train_x, train_y, categorical_feature=categoIdx,
                  eval_set=(val_x,val_y), eval_metric="auc", callbacks=cb_list)
    elif model_name == "XGB_GBT":
        tuning_params = {
            "max_depth": trial.suggest_int("max_depth", 4, 8, step=1),
            "subsample": trial.suggest_float("subsample", 0.5, 0.8, step=0.05),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 0.8, step=0.05),
            "reg_lambda": trial.suggest_float("reg_lambda", 0.01, 10.0, step=0.01),
            "min_child_weight": trial.suggest_float("min_child_weight", 0.01, 10.0, step=0.01),
            "gamma": trial.suggest_float("gamma", 0.01, 10.0, step=0.01),
            "max_delta_step":  trial.suggest_float("max_delta_step", 0.01, 10.0, step=0.01)
        }
        model = xgb.XGBRanker(booster="gbtree", objective="rank:ndcg",
                            tree_method="gpu_hist", sampling_method="gradient_based",
                            n_estimators=int(ntrees * 0.2),
                            learning_rate=eta / 10,
                            random_state=fold, verbosity=0,
                            **tuning_params)
        model.fit(train_x, train_y, group=train_groups, verbose=False)
    elif model_name == "CAT_GBM":
        tuning_params = {
            # "n_estimators" : trial.suggest_int("n_estimators", 500, 5000, step=500),
            # "learning_rate" : trial.suggest_categorical("learning_rate", [1e-3, 5e-3, 1e-2, 5e-2, 1e-1]),
            "max_depth": trial.suggest_int("max_depth", 4, 8, step=1),
            "bagging_temperature": trial.suggest_float("bagging_temperature", 0.05, 0.95, step=0.05),
            # "rsm": trial.suggest_float("rsm", 0.5, 0.9, step=0.05),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.01, 10.0, step=0.01),
            "random_strength": trial.suggest_float("random_strength", 0.01, 1.5, step=0.01),
            "min_child_samples": trial.suggest_int("min_child_samples", 3, 99, step=2),  
            "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1.0, 3.0, step=0.01)
        }

        model = cat.CatBoostClassifier(boosting_type="Plain", loss_function="MultiClass", task_type="GPU",
                                    n_estimators=int(ntrees * 0.2),
                                    learning_rate=eta / 10,
                                    one_hot_max_size=3, leaf_estimation_method="Gradient",
                                    # leaf_estimation_iterations=5,
                                    # max_ctr_complexity=2,
                                    logging_level="Silent", random_state=fold, thread_count=cpu_count(),
                                    **tuning_params)
        model.fit(train_x, train_y, cat_features=categoIdx)
    elif model_name == "CAT_ORD":
        tuning_params = {
            # "n_estimators" : trial.suggest_int("n_estimators", 500, 5000, step=500),
            # "learning_rate" : trial.suggest_categorical("learning_rate", [1e-3, 5e-3, 1e-2, 5e-2, 1e-1]),
            "max_depth": trial.suggest_int("max_depth", 4, 8, step=1),
            "bagging_temperature": trial.suggest_float("bagging_temperature", 0.05, 0.95, step=0.05),
            # "rsm": trial.suggest_float("rsm", 0.5, 0.9, step=0.05),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.01, 10.0, step=0.01),
            "random_strength": trial.suggest_float("random_strength", 0.01, 1.5, step=0.01),
            "min_child_samples": trial.suggest_int("min_child_samples", 3, 99, step=2),
            "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1.0, 3.0, step=0.01)
        }
        
        model = cat.CatBoostClassifier(boosting_type="Ordered", loss_function="MultiClass", task_type="GPU",
                                    n_estimators=int(ntrees * 0.2),
                                    learning_rate=eta / 10,
                                    one_hot_max_size=3, leaf_estimation_method="Gradient",
                                    # leaf_estimation_iterations=5,
                                    # max_ctr_complexity=2,
                                    logging_level="Silent", random_state=fold, thread_count=cpu_count(),
                                    **tuning_params)
        model.fit(train_x, train_y, cat_features=categoIdx)
    else:
        print("unknown")
        return -1
    
    pred = model.predict_proba(val_x)
    # f1 score
    optuna_score = metrics.f1_score(val_y.tolist(), [1.0 if i > threshold else 0.0 for i in pred[:,1]])

    if optuna_score > output_container["score"]:
        output_container["model"] = model
        output_container["pred"] = pred
        output_container["score"] = optuna_score

    return optuna_score

# Training

In [ ]:
architecture_name = "vggish_lgb_goss_thresholdOri_tuningPosWeight_try1"
createFolder(folder_path + "architecture/" + architecture_name)

In [ ]:
def do_fold_training(fold, train_idx, val_idx, sample_weight=None, finetuning=False):
    tmp_time = time()
    print("\n===== Fold", fold, "=====\n")
    global val_pred, test_pred

    wandb.init(
        project="dacon_covid19_classification",
        group=architecture_name,
        name="fold_" + str(fold)
    ); wandb.config.step = 0

    train_x = df_full.iloc[train_idx][num_vars + bin_vars + audio_vars].copy()
    train_y = df_full.iloc[train_idx][target_var].copy()
    
    val_x = df_full.iloc[val_idx][num_vars + bin_vars + audio_vars].copy()
    val_y = df_full.iloc[val_idx][target_var].copy()
    
    output_container = {"model": None, "pred": None, "score": -np.inf}
    optuna_timout = int(6 * 3600 / kfolds_spliter.get_n_splits())
    optuna_study = create_study(direction='maximize', sampler=TPESampler())
    optuna_study.optimize(
        lambda trial: optuna_objective_function(
            trial, fold, train_x, train_y, None, val_x, val_y, None, categoIdx=None, model_name="LGB_GOSS", output_container=output_container, ntrees=ntrees, eta=eta
        ),
        n_jobs=1, n_trials=300, timeout=optuna_timout
    )
    
    model_list.append(output_container["model"])
    params_list.append(optuna_study.best_params)
    print("fold", fold, "best params :", params_list[-1])
    val_pred[val_idx] = output_container["pred"]
    fold_metric.append(output_container["score"])
    print("fold", fold, "score :", fold_metric[-1])

    test_x = df_test[num_vars + bin_vars + audio_vars].copy()
    test_pred += model_list[-1].predict_proba(test_x) / n_folds

    wandb.log({"fold": fold,
               "logloss": metrics.log_loss(val_y.tolist(), val_pred[val_idx, 1]),
               "auc": metrics.roc_auc_score(val_y.tolist(), val_pred[val_idx, 1]),
               "f1_score": fold_metric[-1]})
    wandb.finish()
 
    print("fold", fold, "time to training :", round(time() - tmp_time, 3))

In [ ]:
# learning parameter setting
# The range of converged trees is about 2000 ~ 4000
ntrees = 5000
eta = 5e-3

# ntrees = 1000
# eta = 1e-2

threshold = round(df_full[target_var].mean(), 5)
# threshold = 0.5

In [ ]:
model_list = []
params_list = []
fold_metric = []

val_pred = np.zeros(shape=(df_full.shape[0], 2))
test_pred = np.zeros(shape=(df_test.shape[0], 2))
seed_everything()

n_folds = 5
kfolds_spliter = StratifiedKFold(n_folds, shuffle=True, random_state=42)

start_time_training = time()
# fold training
for fold, (train_idx, val_idx) in enumerate(kfolds_spliter.split(df_full, y=df_full[target_var])):
    start_mem = memory_usage()   
    do_fold_training(fold, train_idx, val_idx, None, finetuning=False)
    gc.collect()
    end_mem = memory_usage()
    print("@Memory leaked :", end_mem - start_mem, "\n")
end_time_training = time()

[debug] memory usage:    746.559 MB

===== Fold 0 =====



wandb: Currently logged in as: frony. Use `wandb login --relogin` to force relogin


[I 2022-06-16 04:21:35,078] A new study created in memory with name: no-name-2c51b461-fff4-4f95-aa28-9ec6ab3251ad
[I 2022-06-16 04:21:50,690] Trial 0 finished with value: 0.1794871794871795 and parameters: {'num_leaves': 15, 'subsample': 0.8, 'colsample_bytree': 0.65, 'reg_lambda': 0.02589094297065214, 'min_child_weight': 0.6880660203135626, 'min_child_samples': 17, 'scale_pos_weight': 1.107691836763628}. Best is trial 0 with value: 0.1794871794871795.
[I 2022-06-16 04:22:11,505] Trial 1 finished with value: 0.1809045226130653 and parameters: {'num_leaves': 127, 'subsample': 0.75, 'colsample_bytree': 0.5, 'reg_lambda': 0.02386718429551674, 'min_child_weight': 1.1668092599252171, 'min_child_samples': 18, 'scale_pos_weight': 0.5432022857929192}. Best is trial 1 with value: 0.1809045226130653.
[I 2022-06-16 04:22:25,046] Trial 2 finished with value: 0.1884498480243161 and parameters: {'num_leaves': 255, 'subsample': 0.75, 'colsample_bytree': 0.7, 'reg_lambda': 0.10540546892906015, 'min_ch

In [ ]:
avg_score = 0
for idx, value in enumerate(fold_metric):
    print("fold", idx, "score :", value)
    avg_score += value / n_folds
print("average score :", avg_score)

# Submission

In [ ]:
dataframe(val_pred).to_csv(folder_path + "architecture/" + architecture_name + "/valPred_" + architecture_name + ".csv", index=False)
dataframe(test_pred).to_csv(folder_path + "architecture/" + architecture_name + "/testPred_" + architecture_name + ".csv", index=False)

In [ ]:
submission = pd.read_csv(folder_path + 'open.zip (Unzipped Files)/sample_submission.csv')
submission['covid19'] = [1 if i > threshold else 0 for i in test_pred[:,1]]
submission.to_csv(folder_path + "architecture/" + architecture_name + "/submission_" + architecture_name + ".csv", index=False)